從《Python 網路爬蟲與資料分析入門實戰》第三章的範例中練習爬蟲

從第三章奇摩電影爬蟲範例中練習自己寫的程式碼

書中原始程式碼來源：https://github.com/jwlin/web-crawler-tutorial/tree/master/ch3

# 本周新片：(1)函數爬蟲資料並儲存(2)爬蟲兩頁的資料並儲存

## 1-1爬蟲函數：將奇摩本周新片的資料抓下來

In [1]:
import requests
import re
from bs4 import BeautifulSoup

url='https://movies.yahoo.com.tw/movie_thisweek.html'

def yahoo_newmovie(url):
    #1.與網頁溝通/連線(使用requests套件)
    resp = requests.get(url)
    #2.剖析網頁原始碼(使用BeautifulSoup剖析器) 
    soup = BeautifulSoup(resp.text, 'html5lib')
    #3.取得網頁內容：3.1電影主資訊的位置
    rows = soup.find_all('div', 'release_info_text')
        #3.2.創造空清單變數存放所有資訊
    movies = []
        #3.3.迴圈跑電影資訊的內容
    for row in rows:
        #3.4.創造空的字典存放每個電影資訊
        movie = dict()
        #3.5.創造中文電影名稱存放電影的「中文名稱」:div標籤/release_movie_name屬性/a標籤
        movie['ch_name'] = row.find('div', 'release_movie_name').a.text.strip()
        #3.5.創造「英文名稱」:div標籤/release_movie_name屬性中的div標籤/en屬性/a標籤
        movie['eng_name'] = row.find('div', 'release_movie_name').find('div', 'en').a.text.strip()
            ##3.5.1.創造符合上映日期的格式(三組一或多個數字)
        pattern = '\d+-\d+-\d+'
            ##3.5.2.將上映日期位置(div標籤/release_movie_time屬性)的內容放入匹配格式的re.search套件中
        match = re.search(pattern,row.find('div', 'release_movie_time').text)
            ##3.5.2創造「上映日期」:將符合格式的內容放入
        movie['release_date'] = match.group(0)
        #3.6創造「期待值」:div標籤/leveltext屬性/span標籤
        movie['expectation'] = row.find('div', 'leveltext').span.text.strip()
        #3.7創造「電影網址」:div標籤/release_movie_name屬性/a標籤的href
        movie['url'] = row.find('div', 'release_movie_name').a['href']
        #3.8創造「電影ID」:從電影網址中去除.html,再分隔-取出最右邊的電影ID
        movie['movie_id'] =movie['url'].split('.html')[0].split('-')[-1]
        #3.9創造「預告網址」:主資訊的父節點的前一個兄弟節點的div標籤/release_foto屬性/a標籤的img的src
        movie['poster_url'] = row.parent.find_previous_sibling('div', 'release_foto').a.img['src']
        #3.10t創造「簡介」:div標籤/release_text屬性並取代\n
        movie['intro'] = row.find('div', 'release_text').text.replace('\n', '').strip()
        #3.11創造變項trailer_a放入預告片網址位置:div標籤/release_btn color_btnbox屬性的所有第二個a標籤
        trailer_a = row.find_next_sibling('div', 'release_btn color_btnbox').find_all('a')[1]
            #3.11.1創造「預告片網址」：如果href有在變項中就放入變項的href內容，沒有則填入空白
        movie['trailer_url'] = trailer_a['href'] if 'href' in trailer_a.attrs.keys() else ''
        #3.12將上述資料添加到movies中
        movies.append(movie)
        #3.13返回所有電影資訊的變項
    return movies
yahoo_newmovie(url)

[{'ch_name': '賽道狂人',
  'eng_name': 'Ford v Ferrari',
  'expectation': '78%',
  'intro': '★集結影壇夢幻卡司！麥特戴蒙聯手克里斯汀貝爾 重現賽車史經典戰役★年度熱血鉅作《賽道狂人》爛番茄滿分開盤 多倫多國際影展好評熱推★《羅根》才華大導演詹姆士曼格擔崗 媒體盛譽將橫掃獎季★改編自車壇最傳奇熱血的真實事蹟 福特汽車對抗賽車界霸主法拉利\xa0曾獲奧斯卡殊榮的金獎得主麥特戴蒙和金球獎影帝克里斯汀貝爾即將共同出演《賽道狂人》，電影根據真實事件改編，劇情描述來自美國的汽車設計師卡洛謝爾比（麥特戴蒙 飾演）和無所畏懼的英國賽車手肯邁爾斯（克里斯汀貝爾 飾演），兩人聯手對抗企業干預、打破物理定律，同時面對他們各自的心魔，最後為福特汽車打造出一輛革命性新款賽車，更在1966年於法國舉辦的利曼24小時耐力賽中，一舉擊敗當時的賽車界霸主法拉利。',
  'movie_id': '10097',
  'poster_url': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/November2019/O6jhWHvEHNgqvSmQlHH3-800x1142.jpg',
  'release_date': '2019-11-28',
  'trailer_url': 'https://movies.yahoo.com.tw/video/%E8%B3%BD%E9%81%93%E7%8B%82%E4%BA%BA-%E6%9C%80%E6%96%B0%E5%AE%98%E6%96%B9%E9%A0%90%E5%91%8A-091854179.html?movie_id=10097',
  'url': 'https://movies.yahoo.com.tw/movieinfo_main/%E8%B3%BD%E9%81%93%E7%8B%82%E4%BA%BA-ford-v-ferrari-10097'},
 {'ch_name': '鋒迴路轉',
  'eng_name': 'Knives Out',
  'expectation': '87%',
  'intro': '★ 《STAR WARS：最後的絕地武士》原班團隊犯罪懸疑鉅

##  1-2整理爬下來的資料(dataframe)並印出資料筆數與期待值高的電影

In [2]:
import pandas as pd
import numpy as np

#4.將本周新片整理成dataframe,印出符合條件的電影並儲存成csv檔
    #4.1用函數取出所有資料
data=yahoo_newmovie(url)
    #4.2命名各個欄位順序(不命名則會照開頭字母順序)
name=['ch_name','eng_name','release_date','expectation','url','movie_id','poster_url','intro','trailer_url']
    #4.3將資料轉成dataframe
test=pd.DataFrame(columns=name,data=data)
    #4.4整理資料中的索引(從1開始編號)
test.index = np.arange(1,len(test)+1)
    #4.5將索引重新命名成電影編號
test.index.names = ['movie_NO.']
    #4.6顯示資料量(共有幾部電影)
print('本周新片', len(data), '部電影')
    #4.7顯示期待值超過90%的電影名稱
print('期待值> 90% :' )
n='90%'
for a in data:
    if a['expectation'] > n :
        print(a['ch_name'],a['expectation'])

本周新片 9 部電影
期待值> 90% :
布魯克林孤兒 91%
懸案密碼前傳：瓶中信 92%
夕霧花園 95%


## 1-3印出所有資料並儲存成csv檔

In [3]:
    #4.8印出所有電影
print('本周新片:',test)
    #4.9儲存成csv檔案
test.to_csv('newmovie.csv',encoding='utf_8_sig')

本周新片:               ch_name                                eng_name release_date  \
movie_NO.                                                                    
1                賽道狂人                          Ford v Ferrari   2019-11-28   
2                鋒迴路轉                              Knives Out   2019-11-28   
3               暴走曼哈頓                              21 Bridges   2019-11-29   
4              布魯克林孤兒                     Motherless Brooklyn   2019-11-29   
5          懸案密碼前傳：瓶中信                   A Conspiracy Of Faith   2019-11-29   
6               寂寞診療室                                   Sibyl   2019-11-29   
7                夕霧花園             The Garden of Evening Mists   2019-11-29   
8            布紐爾超現實人生  Buñuel in the Labyrinth of the Turtles   2019-11-29   
9               怪奇超自然                          Strange Nature   2019-11-29   

          expectation                                                url  \
movie_NO.                                                  

## 2-1若本周新片有兩頁的資料

若本周新片沒有第二頁資料則會Error,先用上映中的電影網址代替

In [4]:
import requests
import re
from bs4 import BeautifulSoup

##用上映中的電影網址做範例
url='https://movies.yahoo.com.tw/movie_intheaters.html'    
#將第二頁網頁的電影內容加入(知道搜尋結果只有兩頁的程式碼)

    #創造空清單變數存放所有資訊
movies=[]
    #用函數爬出第一頁內容
movie=yahoo_newmovie(url)
    #將第一頁內容加入變數中
movies += movie
    #向第一頁搜尋網頁發出請求
resp = requests.get(url)
    #剖析原始碼
soup = BeautifulSoup(resp.text, 'html5lib')
    #如果有下一頁的網址則儲存下一頁的內容
if soup.find('li', 'nexttxt').a['href']:
    #創造下一頁的網址變數
    nextpage = soup.find('li', 'nexttxt').a['href']
    #將下一頁內容爬出來
    new_movie=yahoo_newmovie(nextpage)
    #將新的內容儲存至清單變數中
    movies += new_movie
        
import pandas as pd
import numpy as np

#將本周新片整理成dataframe,印出符合條件的電影並儲存成csv檔
    #用函數取出所有資料
data=movies
    #命名各個欄位順序(不命名則會照開頭字母順序)
name=['ch_name','eng_name','release_date','expectation','url','movie_id','poster_url','intro','trailer_url']
    #將資料轉成dataframe
test=pd.DataFrame(columns=name,data=data)
    #整理資料中的索引(從1開始編號)
test.index = np.arange(1,len(test)+1)
    #將索引重新命名成電影編號
test.index.names = ['movie_NO.']
    #顯示資料量(共有幾部電影)
print('本周新片', len(data), '部電影')
    #顯示期待值超過90%的電影名稱
print('期待值> 90% :' )
n='90%'
for a in data:
    if a['expectation'] > n :
        print(a['ch_name'],a['expectation'])
    #4.8印出所有電影
print('本周新片:',test)
    #4.9儲存成csv檔案
test.to_csv('newmovie.csv',encoding='utf_8_sig')

本周新片 20 部電影
期待值> 90% :
冰雪奇緣2 92%
神機有毛病 95%
萬萬沒想到 97%
本周新片:                          ch_name                     eng_name release_date  \
movie_NO.                                                                    
1                           小小夜曲   Little Nights, Little Love   2019-11-22   
2          BanG Dream！ FILM LIVE         BanG Dream！FILM LIVE   2019-11-22   
3                            許怨房                     The Room   2019-11-22   
4                       82年生的金智英      Kim Ji-Young Born, 1982   2019-11-22   
5                          改革好萊塢      This Changes Everything   2019-11-22   
6                        女神們的下午茶          Nothing Like a Dame   2019-11-22   
7                            暗影人               The Shadow Man   2019-11-22   
8                           野蠻迷林                       Savage   2019-11-22   
9                        麂皮：永不滿足   Suede: The Insatiable Ones   2019-11-22   
10                         男人真命苦                     Miyamoto   2019-11-22   
11   

# 無註解程式碼

In [ ]:
import requests
import re
from bs4 import BeautifulSoup

url='https://movies.yahoo.com.tw/movie_thisweek.html'

def yahoo_newmovie(url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html5lib')
    rows = soup.find_all('div', 'release_info_text')
    movies = []
    for row in rows:
        movie = dict()
        movie['ch_name'] = row.find('div', 'release_movie_name').a.text.strip()
        movie['eng_name'] = row.find('div', 'release_movie_name').find('div', 'en').a.text.strip()
        pattern = '\d+-\d+-\d+'
        match = re.search(pattern,row.find('div', 'release_movie_time').text)
        movie['release_date'] = match.group(0)
        movie['expectation'] = row.find('div', 'leveltext').span.text.strip()
        movie['url'] = row.find('div', 'release_movie_name').a['href']
        movie['movie_id'] =movie['url'].split('.html')[0].split('-')[-1]
        movie['poster_url'] = row.parent.find_previous_sibling('div', 'release_foto').a.img['src']
        movie['intro'] = row.find('div', 'release_text').text.replace('\n', '').strip()
        trailer_a = row.find_next_sibling('div', 'release_btn color_btnbox').find_all('a')[1]
        movie['trailer_url'] = trailer_a['href'] if 'href' in trailer_a.attrs.keys() else ''
        movies.append(movie)
    return movies

import pandas as pd
import numpy as np

data=yahoo_newmovie(url)
name=['ch_name','eng_name','release_date','expectation','url','movie_id','poster_url','intro','trailer_url']
test=pd.DataFrame(columns=name,data=data)
test.index = np.arange(1,len(test)+1)
test.index.names = ['movie_NO.']

print('本周新片', len(data), '部電影')
print('期待值> 90% :' )
n='90%'
for a in data:
    if a['expectation'] > n :
        print(a['ch_name'],a['expectation'])
        

print('本周新片:',test)
test.to_csv('newmovie.csv',encoding='utf_8_sig')

In [ ]:
import requests
import re
from bs4 import BeautifulSoup

url='https://movies.yahoo.com.tw/movie_thisweek.html'    

movies=[]
movie=yahoo_newmovie(url)
movies += movie
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html5lib')

if soup.find('li', 'nexttxt').a['href']:
    nextpage = soup.find('li', 'nexttxt').a['href']
    new_movie=yahoo_newmovie(nextpage)
    movies += new_movie
        
        
import pandas as pd
import numpy as np

data=movies
name=['ch_name','eng_name','release_date','expectation','url','movie_id','poster_url','intro','trailer_url']
test=pd.DataFrame(columns=name,data=data)
test.index = np.arange(1,len(test)+1)
test.index.names = ['movie_NO.']

print('本周新片', len(data), '部電影')

print('期待值> 90% :' )
n='90%'
for a in data:
    if a['expectation'] > n :
        print(a['ch_name'],a['expectation'])
        
print('本周新片:',test)

test.to_csv('newmovie.csv',encoding='utf_8_sig')